# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Import packages

In [1]:
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [10]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':float, 
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype = dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [11]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
#sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [12]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [13]:
from sklearn import linear_model
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights
feature_select_all = pd.DataFrame({'features': all_features,
                                   'weights': model_all.coef_})

Find what features had non-zero weight.

In [14]:
feature_select_all

,features,weights
0,bedrooms,0.000000
1,bedrooms_square,0.000000
2,bathrooms,0.000000
3,sqft_living,134.439314
4,sqft_living_sqrt,0.000000
5,sqft_lot,0.000000
6,sqft_lot_sqrt,0.000000
7,floors,0.000000
8,floors_square,0.000000
9,waterfront,0.000000


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [15]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [16]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [17]:
RSS_valid = pd.DataFrame(columns=['l1_penalty', 'RSS'])
index = 0
for l1_penalty in np.logspace(1, 7, num=13):
    #Learn a model on TRAINING data using the specified l1_penalty.
    lml = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    lml.fit(training[all_features], training['price'])
    RSS=sum((lml.predict(validation[all_features])-validation['price'])**2)
    RSS_valid.loc[index] = [l1_penalty, RSS]
    index += 1

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [22]:
l1_penalty_optimal = RSS_valid[RSS_valid['RSS']==min(RSS_valid['RSS'])
                               ]['l1_penalty'] #return a series with index of 0
lpo = np.array(l1_penalty_optimal) #return array([10.])
#lpo = np.array(l1_penalty_optimal[0]) #return array(10.)
print('The best value for the l1_penalty: ' + str(lpo[0]))
#lpo = l1_penalty_optimal[0]
#print('The best value for the l1_penalty: {:.4}'.format(lpo))

The best value for the l1_penalty: 10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [26]:
model_optimal = linear_model.Lasso(alpha=lpo, normalize=True)
model_optimal.fit(training[all_features], training['price'])
nonzero_weights = np.count_nonzero(model_optimal.coef_) + \
                  np.count_nonzero(model_optimal.intercept_)
print(nonzero_weights)
feature_optimal = pd.DataFrame({'features': all_features,
                                'weights': model_optimal.coef_})

15


In [27]:
feature_optimal

,features,weights
0,bedrooms,-16144.562757
1,bedrooms_square,373.245384
2,bathrooms,50841.243340
3,sqft_living,617.853560
4,sqft_living_sqrt,-44411.354867
5,sqft_lot,0.785623
6,sqft_lot_sqrt,-701.194765
7,floors,-0.000000
8,floors_square,5014.200457
9,waterfront,619488.752486


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [45]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [44]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [41]:
#Search for best value for l1 penalty with a for loop
def searchForOptimal(x1, y1, x2, y2, penalties, str1, str2, str3): 
    RSS_valid = pd.DataFrame(columns=[str1, str2, str3])
    index = 0
    for l1_penalty in penalties:
        #Learn a model on TRAINING data using the specified l1_penalty.
        model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
        model.fit(x1, y1)
        count_nonzero = np.count_nonzero(model.coef_) +\
                        np.count_nonzero(model.intercept_)
        RSS = ((model.predict(x2) - y2)**2).sum()
        RSS_valid.loc[index] = [l1_penalty, RSS, count_nonzero]
        index += 1    
        print(index, l1_penalty, count_nonzero)
    return RSS_valid

In [47]:
#use np.logspace(1, 4, num=20) in order to get close to graphlab
RSS_valid_wide = searchForOptimal(training[all_features], training['price'], 
                                  validation[all_features], 
                                  validation['price'],
                                  np.logspace(1, 4, num=20), 'l1_penalty', 
                                  'RSS', 'count_nonzero') 

1 10.0 15
2 14.38449888287663 15
3 20.6913808111479 15
4 29.76351441631318 15
5 42.81332398719393 13
6 61.58482110660264 12
7 88.58667904100822 11
8 127.42749857031335 10
9 183.29807108324357 7
10 263.6650898730358 6
11 379.26901907322497 6
12 545.5594781168514 6
13 784.7599703514607 5
14 1128.8378916846884 3
15 1623.776739188721 3
16 2335.7214690901214 2
17 3359.818286283781 1
18 4832.930238571752 1
19 6951.927961775606 1
20 10000.0 1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [57]:
#lower bound for l1 penalty; otherwise, too many features selected
l1_penalty_min=RSS_valid_wide[RSS_valid_wide['count_nonzero']>max_nonzeros]['l1_penalty'].max()

#upper bound for l1 penalty; otherwise, too few features selected
l1_penalty_max=RSS_valid_wide[RSS_valid_wide['count_nonzero']<max_nonzeros]['l1_penalty'].min()

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [58]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [59]:
RSS_valid_narrow = searchForOptimal(training[all_features], training['price'], 
                                    validation[all_features], 
                                    validation['price'],
                                    l1_penalty_values, 'l1_penalty', 
                                    'RSS', 'count_nonzero') 

1 127.42749857031335 10
2 134.5978981125619 10
3 141.76829765481045 8
4 148.938697197059 8
5 156.10909673930755 7
6 163.2794962815561 7
7 170.44989582380464 7
8 177.6202953660532 7
9 184.79069490830176 7
10 191.96109445055032 7
11 199.13149399279888 7
12 206.3018935350474 6
13 213.47229307729594 6
14 220.6426926195445 6
15 227.81309216179307 6
16 234.98349170404163 6
17 242.1538912462902 6
18 249.32429078853872 6
19 256.49469033078725 6
20 263.6650898730358 6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [82]:
RSS_valid_narrow_min = RSS_valid_narrow[RSS_valid_narrow['count_nonzero']==max_nonzeros]['RSS'].min()
l1_penalty_narrow_opt=RSS_valid_narrow[RSS_valid_narrow['RSS']==RSS_valid_narrow_min]['l1_penalty'] 
#return a series with index of 4
print('Value of l1_penalty in the narrow range has the lowest RSS on the VALIDATION set and sparsity equal to max_nonzeros: \
{}'.format(l1_penalty_narrow_opt[4]))

Value of l1_penalty in the narrow range has the lowest RSS on the VALIDATION set and sparsity equal to max_nonzeros: 156.10909673930755


In [86]:
#best value for l1 penalty within the narrow range
lpon = np.array(l1_penalty_narrow_opt[4])
model_lpon = linear_model.Lasso(alpha = lpon, normalize = True)
model_lpon.fit(training[all_features], training['price'])
feature_lpon = pd.DataFrame({'features': all_features,
                             'weights': model_lpon.coef_})
feature_lpon

,features,weights
0,bedrooms,-0.000000
1,bedrooms_square,-0.000000
2,bathrooms,10610.890284
3,sqft_living,163.380252
4,sqft_living_sqrt,0.000000
5,sqft_lot,-0.000000
6,sqft_lot_sqrt,-0.000000
7,floors,0.000000
8,floors_square,0.000000
9,waterfront,506451.687115


In [88]:
#return l1 penalty min having nonzeros > max_nonzeros > l1 penalty max having nonzeros
l1_penalty_min = 127.42749857031335
l1_penalty_max = 183.29807108324357
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
RSS_valid_narrow = searchForOptimal(training[all_features], training['price'], 
                                    validation[all_features], 
                                    validation['price'],
                                    l1_penalty_values, 'l1_penalty', 
                                    'RSS', 'count_nonzero') 
RSS_valid_narrow_min = RSS_valid_narrow[RSS_valid_narrow['count_nonzero']==max_nonzeros]['RSS'].min()
l1_penalty_narrow_opt_row=RSS_valid_narrow[RSS_valid_narrow['RSS']==RSS_valid_narrow_min]
print(l1_penalty_narrow_opt_row)

1 127.42749857031335 10
2 130.3680550183623 10
3 133.30861146641126 10
4 136.24916791446023 9
5 139.1897243625092 8
6 142.13028081055813 8
7 145.0708372586071 8
8 148.01139370665607 8
9 150.95195015470503 8
10 153.89250660275397 7
11 156.83306305080293 7
12 159.7736194988519 7
13 162.71417594690087 7
14 165.6547323949498 7
15 168.59528884299877 7
16 171.53584529104774 7
17 174.4764017390967 7
18 177.41695818714567 7
19 180.3575146351946 7
20 183.29807108324357 7
   l1_penalty           RSS  count_nonzero
9  153.892507  4.398200e+14            7.0


In [89]:
lpon = np.array(l1_penalty_narrow_opt_row.iloc[:,0])
model_lpon = linear_model.Lasso(alpha = lpon, normalize = True)
model_lpon.fit(training[all_features], training['price'])
feature_lpon = pd.DataFrame({'features': all_features,
                             'weights': model_lpon.coef_})
feature_lpon

,features,weights
0,bedrooms,-0.000000
1,bedrooms_square,-0.000000
2,bathrooms,11074.253516
3,sqft_living,163.235168
4,sqft_living_sqrt,0.000000
5,sqft_lot,-0.000000
6,sqft_lot_sqrt,-0.000000
7,floors,0.000000
8,floors_square,0.000000
9,waterfront,508217.547491


In [93]:
print('Value of l1_penalty in the narrow range has the lowest RSS on the VALIDATION set and sparsity equal to max_nonzeros: \
{}'.format(lpon[0]))

Value of l1_penalty in the narrow range has the lowest RSS on the VALIDATION set and sparsity equal to max_nonzeros: 153.89250660275397
